# Speech to Text with Whisper OpenAI

### Import Dependencies

In [1]:
from pydub import AudioSegment
import whisper
from tqdm.notebook import tqdm
import os
import warnings
from datetime import timedelta, datetime
import textwrap
from pydub.utils import mediainfo
import platform
import psutil
import time
import json
import hashlib


In [2]:
# --- Siapkan path file unik ---
def get_unique_filename(base_path):
    if not os.path.exists(base_path):
        return base_path
    name, ext = os.path.splitext(base_path)
    counter = 1
    while True:
        new_path = f"{name}_{counter}{ext}"
        if not os.path.exists(new_path):
            return new_path
        counter += 1

### Load Model & Estimate

In [3]:
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")

# --- Load Whisper model ---
print("🔧 Load Whisper model...")
model = whisper.load_model("turbo") 
print("✅ Whisper model loaded.")

🔧 Load Whisper model...
✅ Whisper model loaded.


In [4]:
# --- Load file dari subfolder ---
def pilih_subfolder():
    print("📁 Pilih subfolder:")
    subfolders = ['audio', 'video']
    for i, sf in enumerate(subfolders, 1):
        print(f"   {i}. {sf}")
    while True:
        try:
            pilihan = int(input("Masukkan nomor subfolder: "))
            if 1 <= pilihan <= len(subfolders):
                return subfolders[pilihan - 1]
        except ValueError:
            pass
        print("❌ Pilihan tidak valid. Coba lagi.")

def pilih_file(folder_path):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if not files:
        print("🚫 Tidak ada file ditemukan.")
        exit()
    print(f"🎧 Pilih file dari '{os.path.basename(folder_path)}':")
    for i, f in enumerate(files, 1):
        print(f"   {i}. {f}")
    while True:
        try:
            pilihan = int(input("Masukkan nomor file: "))
            if 1 <= pilihan <= len(files):
                return files[pilihan - 1]
        except ValueError:
            pass
        print("❌ Pilihan tidak valid. Coba lagi.")

# --- Proses utama ---
base_path = "assets"
subfolder = pilih_subfolder()
folder_path = os.path.join(base_path, subfolder)
file_name = pilih_file(folder_path)
audio_path = os.path.join(folder_path, file_name)

print(f"🎵 Load audio file: {audio_path}")
audio = AudioSegment.from_file(audio_path)


# --- Extract and display metadata ---
info = mediainfo(audio_path)
duration_sec = float(info.get("duration", 0))
duration_str = str(timedelta(seconds=int(duration_sec)))

print("ℹ️  Metadata Audio:")
print(f"    • File name    : {audio_path.split('/')[-1]}")
print(f"    • Duration     : {duration_str}")
print(f"    • Codec        : {info.get('codec_name', 'N/A')}")
print(f"    • Sample rate  : {info.get('sample_rate', 'N/A')} Hz")
print(f"    • Channels     : {info.get('channels', 'N/A')}")
print(f"    • Bitrate      : {info.get('bit_rate', 'N/A')} bps")
print(f"    • Format       : {info.get('format_name', 'N/A')}")

print("✅ Audio loaded.")

# --- Split audio into 1-minute chunks ---
chunk_duration_ms = 60 * 1000  # 60 detik
chunks = [audio[i:i + chunk_duration_ms] for i in range(0, len(audio), chunk_duration_ms)]
print(f"📦 Total chunks: {len(chunks)} (durasi: 1 menit per chunk)")

# Info spek sistem
print("\n💻 Spesifikasi Sistem:")
print(f"    • Platform        : {platform.system()} {platform.release()}")
print(f"    • Processor       : {platform.processor()}")
print(f"    • CPU cores       : {psutil.cpu_count(logical=False)} fisik / {psutil.cpu_count(logical=True)} logical")
print(f"    • RAM             : {round(psutil.virtual_memory().total / (1024**3), 2)} GB")

# --- Estimasi waktu adaptif sebelum proses chunk ---
print("⏳ Menghitung estimasi waktu transkripsi adaptif (1 menit)...")

# Ambil 1 menit pertama dari audio (atau kurang kalau audio-nya pendek)
sample_duration_ms = min(60 * 1000, len(audio))  # Maks 1 menit
sample_chunk = audio[:sample_duration_ms]
temp_sample_path = "temp_sample_chunk.wav"
sample_chunk.export(temp_sample_path, format="wav")

# Hitung waktu transkripsi
start_time = time.time()
result = model.transcribe(temp_sample_path, language="id")
chunk_text = result["text"].strip()
elapsed = time.time() - start_time

# Estimasi total
estimated_per_chunk = round(elapsed, 2)
estimated_chunks = int(len(audio) / sample_duration_ms)
estimated_total_time = elapsed * estimated_chunks
estimated_total_fmt = str(timedelta(seconds=int(estimated_total_time)))


print(f"    🧮 Estimasi waktu per chunk   : {estimated_per_chunk} detik")
print(f"    🧮 Estimasi total transkripsi : {estimated_total_fmt} untuk ~{estimated_chunks} chunks")

# Hapus file sementara
if os.path.exists(temp_sample_path):
    os.remove(temp_sample_path)

print("✅ Estimasi selesai.\n")
print(f"📄 Contoh transkripsi chunk sample:\n{textwrap.fill(chunk_text, width=100)}\n")

📁 Pilih subfolder:
   1. audio
   2. video
🎧 Pilih file dari 'audio':
   1. rev_ppt_transformasi_pairen.m4a
   2. ZoomMeeting_SentimentAnalysis.m4a
   3. .DS_Store
   4. FGD peranan unit transformasi organisasi untuk mendukung perubahan disruptif sesi 2.mp3
   5. FGD peranan unit transformasi organisasi untuk mendukung perubahan disruptif sesi 1.mp3
   6. RFOJK20250603_Risalah Rapat Taksonomi Data.m4a
   7. rev_ppt_transformasi_mbaode.m4a
   8. GMT20250704-063827_Recording_1920x1080.mp4
🎵 Load audio file: assets/audio/ZoomMeeting_SentimentAnalysis.m4a
ℹ️  Metadata Audio:
    • File name    : ZoomMeeting_SentimentAnalysis.m4a
    • Duration     : 1:04:09
    • Codec        : aac
    • Sample rate  : 32000 Hz
    • Channels     : 1
    • Bitrate      : 127314 bps
    • Format       : mov,mp4,m4a,3gp,3g2,mj2
✅ Audio loaded.
📦 Total chunks: 65 (durasi: 1 menit per chunk)

💻 Spesifikasi Sistem:
    • Platform        : Darwin 24.5.0
    • Processor       : arm
    • CPU cores       : 8 fisik

### Transcribe

In [12]:
def calculate_file_hash(filepath):
    hash_md5 = hashlib.md5()
    with open(filepath, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# --- Buat folder output jika belum ada ---
output_dir = "output_transkrip"
os.makedirs(output_dir, exist_ok=True)

# --- Cek checkpoint dan validasi file audio ---
checkpoint_file = os.path.join(output_dir, "checkpoint.json")
audio_hash = calculate_file_hash(audio_path)
start_chunk = 0
cleaned_name = os.path.splitext(file_name)[0].lower()

if os.path.exists(checkpoint_file):
    with open(checkpoint_file, "r") as f:
        checkpoint_data = json.load(f)
        saved_hash = checkpoint_data.get("audio_hash")
        if saved_hash == audio_hash:
            start_chunk = checkpoint_data.get("last_processed_chunk", 0)
            output_txt_path = checkpoint_data.get("output_txt_path")
            output_vtt_path = checkpoint_data.get("output_vtt_path")
            print(f"⏸️ Melanjutkan dari chunk ke-{start_chunk + 1}")
        else:
            print("⚠️ File audio berbeda. Mulai dari awal.")
            start_chunk = 0
            today_str = datetime.today().strftime("%Y%m%d")
            output_txt_path = get_unique_filename(os.path.join(output_dir, f"{cleaned_name}_{today_str}.txt"))
            output_vtt_path = get_unique_filename(os.path.join(output_dir, f"{cleaned_name}_{today_str}.vtt"))
else:
    print("🚀 Memulai proses baru dari awal...")
    start_chunk = 0
    today_str = datetime.today().strftime("%Y%m%d")
    output_txt_path = get_unique_filename(os.path.join(output_dir, f"{cleaned_name}_{today_str}.txt"))
    output_vtt_path = get_unique_filename(os.path.join(output_dir, f"{cleaned_name}_{today_str}.vtt"))

# --- Format waktu ---
def format_time(ms):
    return str(timedelta(milliseconds=ms)).split(".")[0]

def format_time_vtt(ms):
    td = timedelta(milliseconds=ms)
    total_seconds = int(td.total_seconds())
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    milliseconds = ms % 1000
    return f"{hours:02}:{minutes:02}:{seconds:02}.{milliseconds:03}"

# --- Siapkan path file unik (jika diperlukan) ---
def get_unique_filename(base_path):
    if not os.path.exists(base_path):
        return base_path
    name, ext = os.path.splitext(base_path)
    counter = 1
    while True:
        new_path = f"{name}_{counter}{ext}"
        if not os.path.exists(new_path):
            return new_path
        counter += 1

# --- Buka file untuk tulis awal jika dari awal, atau append jika melanjutkan ---
txt_mode = "w" if start_chunk == 0 else "a"
vtt_mode = "w" if start_chunk == 0 else "a"

with open(output_txt_path, txt_mode, encoding="utf-8") as txt_file, \
     open(output_vtt_path, vtt_mode, encoding="utf-8") as vtt_file:

    if start_chunk == 0:
        vtt_file.write("WEBVTT\n\n")  # Header hanya ditulis saat awal

    full_transcript = ""

    progress_bar = tqdm(
        total=len(chunks),
        initial=start_chunk,
        desc="🔄 Transcribing",
        unit="chunk",
        leave=True
    )

    for i in range(start_chunk, len(chunks)):
        start_ms = i * chunk_duration_ms
        end_ms = min((i + 1) * chunk_duration_ms, len(audio))
        chunk_path = f"temp_chunk_{i}.wav"
        chunks[i].export(chunk_path, format="wav")

        try:
            result = model.transcribe(chunk_path, language="id")
            chunk_text = result["text"].strip()

            if len(chunk_text.split()) < 3:
                progress_bar.update(1)
                continue

            start_display = format_time(start_ms)
            end_display = format_time(end_ms)
            start_vtt = format_time_vtt(start_ms)
            end_vtt = format_time_vtt(end_ms)

            # --- Tulis ke file ---
            txt_file.write(f"[{start_display} - {end_display}]\n{chunk_text}\n\n")
            vtt_file.write(f"{i+1}\n{start_vtt} --> {end_vtt}\n{chunk_text}\n\n")

            # --- Tampilkan ke console ---
            wrapped_text = textwrap.fill(chunk_text, width=100)
            tqdm.write(f"{i+1}\n{start_vtt} --> {end_vtt}\n{wrapped_text}\n")

            full_transcript += chunk_text + " "

            # ✅ Update checkpoint
            with open(checkpoint_file, "w") as f:
                json.dump({
                    "last_processed_chunk": i + 1,
                    "audio_hash": audio_hash,
                    "output_txt_path": output_txt_path,
                    "output_vtt_path": output_vtt_path
                }, f)

        finally:
            if os.path.exists(chunk_path):
                os.remove(chunk_path)
            progress_bar.update(1)

    progress_bar.close()

# --- Ringkasan ---
print(f"\n💾 Transkrip lengkap disimpan ke file: {output_txt_path}")
print(f"💾 Subtitle disimpan ke file: {output_vtt_path}")

print("\n📄 Transkrip Ringkas (tanpa timestamp):\n")
# print(full_transcript.strip())

⏸️ Melanjutkan dari chunk ke-66


🔄 Transcribing: 100%|##########| 65/65 [00:00<?, ?chunk/s]


💾 Transkrip lengkap disimpan ke file: output_transkrip/transkrip_output_20250718_1.txt
💾 Subtitle disimpan ke file: output_transkrip/transkrip_output_20250718_1.vtt

📄 Transkrip Ringkas (tanpa timestamp):

